In [1]:
import pandas as pd

df=pd.read_csv('invoicedetail.csv' , encoding='latin-1')

df=df[df.dateofcreation>='2020-06-10 12:55:59']

df=df.drop(['pid','brand','invoice_id.1','partner_id','customer_id','dateofcreation'],axis=1)

In [10]:
df.head()

,name,invoice_id,quantity,discount,totalprice,grandtotal
20,Akurit 4 Tablet,3291,60,10.8,161.00,161.0
66,Crocin 1000mg Tablet,1734,3,10.0,12.29,670.0
67,Crocin 1000mg Tablet,1741,2,10.0,8.19,8.0
68,Crocin 1000mg Tablet,1742,1,10.0,4.10,4.0
69,Crocin 1000mg Tablet,1743,4,10.0,16.38,16.0


In [11]:
# Removed Biased fields 
df=df[~df['name'].str.contains("Extras|Amul Gold Full Cream Milk|Amul Taaza Toned Milk|Amul Masti Dahi|Amul Chach Buttermilk|Parle G biscuit 5 Rs|Parle 20 20 Cashew Cookie|Loose Sugar",na=False)]

In [12]:
# Invoice
df['invoice_id'].nunique()

4027

In [5]:
# Brands
df['name'].nunique()

842

In [13]:
df_purchase=df.groupby('invoice_id').name.apply(list).reset_index()
df_purchase.head()

,invoice_id,name
0,1237,[Viman Jeera]
1,1241,"[Haldiram Navrattan Namkeen, Haldiram Aloo Bhu..."
2,1242,"[Catch club soda with spring water, Uncle Chip..."
3,1243,[Harvest Gold Fruit Bread]
4,1244,"[Amul Kool Kesar Flavour, Kurkure Puffcorn Yum..."


In [14]:
# Finding the most frequent occurrences of pairs in a list of lists

sell_products=[]
for sell in df_purchase['name']:
    sell_products.append(sell)


In [15]:
# Finding the most frequent occurrences of pairs in a list of lists
from collections import Counter
from itertools import combinations

occurences = Counter()
for sub in sell_products:
    if len(sell_products) < 2:
        continue
    for comb in combinations(sub,2):
        occurences[comb] += 1

In [17]:
df_number_occurence=pd.DataFrame([occurences])

In [18]:
df_number_occurence=df_number_occurence.T

In [19]:
df_number_occurence[df_number_occurence[0]>10].to_csv('two_item_combo.csv')


In [22]:
# creating datastructure to handle runtime queries


import numpy as np

n= df['name'].nunique()
produts= list(df['name'].unique())

data_structure = np.zeros([n, n], dtype = int)

data_structure

for pair in occurences.most_common():
    i=produts.index(pair[0][0])
    j=produts.index(pair[0][1])
    data_structure[i,j]=pair[1]
    data_structure[j,i]=pair[1]



In [24]:
# Example 


min_freq_threshold = 3
query='Frooti Mango drink'

print("items related to following query: ", query)
ind=produts.index(query)
for i in np.where(data_structure[ind,:] >= min_freq_threshold)[0]:
    print(produts[i],data_structure[ind,i])

items related to following query:  Frooti Mango drink
Fortune Refined  Soya Bean Oil 6
Rajdhani Besan Gram flour Grade-1 3
Maggi 2 Minutes noodles 5
Soft Drink 9
Coca Cola softdrink 3
Coca Cola- softdrink 3
Fortune Premium Kachi Ghani 8
Good Day Butter Cookies 6
Thums up Soft Drink 3
Mountain Dew Soft Drink 11
Harvest Gold white Bread Sabki Bread 5
Lays American Style Cream & Onion chips 3
Kurkure Puffcorn yummy cheese 3
Lays American Style Cream & Onion Chips 8
Loose Basmati rice Tukda 5
TATA Salt 7
ACT II Butter Flavour Crispy Crunch Popcorn 4
Fena Superwash detergent powder Rose & Chandan  4
Haldiram takatak chatpata masala namkeen 3
Shyam G Pasta crunchy & crispy 3
Maaza original mango Drink 8
Britannia Vitarich sandwich Premium Ehite Bread 3
Britannia Vitarich sandwich Premium White Bread 3
Haldiram Bhujia 4
Loose Arhar/Toor Dal 3
Bisleri with added minerals drinking water 19
Mountain Dew Soft Drink New 4
Priyagold Butter delite Namkeen butter biscuits  4


In [25]:
# for all unique items creating a csv file of related items
import csv
with open('innovators.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for col in produts:
        ind=produts.index(col)
        interestIndex=np.where(data_structure[ind,:] >= min_freq_threshold)[0]
        interestList=[col]
        for i in np.where(data_structure[ind,:] >= min_freq_threshold)[0]:
            interestList.append(produts[i])
        print(interestList)
        writer.writerow(interestList)

['Akurit 4 Tablet']
['Crocin 1000mg Tablet']
['Crocin 120 Suspension Strawberry']
['Volini 100mg Tablet SR']
['Little Hearts Biscuits']
['Powder - Haldi']
['Bambino Vermicelli Roasted']
['Fortune Refined  Soya Bean Oil', 'Rajdhani Sooji Coarse granules', 'MDH Garam Masala', 'Britannia Marie Gold Biscuits', 'Maggi 2 Minutes noodles', 'Soft Drink', 'Lays India Magic Masala chips', 'Coca Cola softdrink', 'Mountain Dew Soft Drink', 'Milkfood Rich desi Ghee', 'MDH Meat Masala', 'Kurkure Masala Munch', 'Frooti Mango drink', 'TATA Salt', 'Everest Sabji Masala', 'shiv pujari jeera', 'Eagle Premium haldi Powder', 'Fena Superwash detergent powder Rose & Chandan ', 'Ghadi detergent Cake', 'Vim dishwash Bar with power of lemons Pack of 4', 'Britannia Vitarich sandwich Premium Ehite Bread', 'Loose Arhar/Toor Dal', 'Loose Masoor Dal', 'Loose Rajma', 'Bisleri with added minerals drinking water']
['Tata PremiumTea desh ki chai']
['Dettol Original Soap (75gm Each)', 'Aashirvaad Shudh Chakki Atta', 'Mou

IndexError: index 842 is out of bounds for axis 0 with size 842